In [3]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.demo_db")

spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.demo_db.sales (
    order_id INT,
    customer STRING,
    amount DOUBLE
) USING iceberg
""")

print("✅ Table 'nessie.demo_db.sales' created.")


✅ Table 'nessie.demo_db.sales' created.


In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg-Nessie-MinIO-Demo")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://warehouse/")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

print("✅ SparkSession connected to Nessie + MinIO via Iceberg.")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/21 07:18:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ SparkSession connected to Nessie + MinIO via Iceberg.


In [4]:
data = [
    (1, "Alice", 100.5),
    (2, "Bob", 230.75),
    (3, "Charlie", 315.2)
]
df = spark.createDataFrame(data, ["order_id", "customer", "amount"])

df.writeTo("nessie.demo_db.sales").append()

print("✅ Data written successfully!")


25/10/21 06:56:27 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to demo_db/sales_806d5e7f-1618-408f-8a27-5f2c4e83847e/data/00007-7-45d5d560-e5d2-452a-99ff-24ab6edf3ce3-0-00001.parquet. This is unsupported
                                                                                

✅ Data written successfully!


In [5]:
spark.sql("SELECT * FROM nessie.demo_db.sales").show()


+--------+--------+------+
|order_id|customer|amount|
+--------+--------+------+
|       1|   Alice| 100.5|
|       2|     Bob|230.75|
|       3| Charlie| 315.2|
+--------+--------+------+



25/10/21 06:57:44 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-040921c4-1a09-4451-b6bc-7957b4eecf18/userFiles-e029a435-891e-40d6-9645-2fe90f71c1f9. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-040921c4-1a09-4451-b6bc-7957b4eecf18/userFiles-e029a435-891e-40d6-9645-2fe90f71c1f9
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.SparkEnv.stop(SparkEnv.scala:108)
	at org.apache.spark.SparkContext.$anonfun$stop$25(SparkContext.scala:2305)
	at org.apac

In [10]:
import requests
resp = requests.get("http://nessie:19120/api/v1/config")
print(resp.status_code)
print(resp.json())


200
{'defaultBranch': 'main', 'maxSupportedApiVersion': 2}


In [14]:
import os
from pynessie import NessieClient
from pynessie.conf import build_config

# 1. Khai báo endpoint Nessie (container trong cùng network)
os.environ["NESSIE_ENDPOINT"] = "http://nessie:19120/api/v1"

# 2. Tạo đối tượng config từ environment
config = build_config()

# 3. Truyền config này vào NessieClient
client = NessieClient(config)

# 4. Kiểm tra kết nối
print("✅ Connected to Nessie")
print("Current default branch:", client.get_default_branch())


✅ Connected to Nessie
Current default branch: main


In [15]:
data2 = [
    (4, "David", 420.0),
    (5, "Eva", 180.5)
]
spark.createDataFrame(data2, ["order_id", "customer", "amount"])\
    .writeTo("nessie.demo_db.sales").append()

spark.sql("SELECT * FROM nessie.demo_db.sales").show()


25/10/21 07:31:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/10/21 07:31:03 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to demo_db/sales_806d5e7f-1618-408f-8a27-5f2c4e83847e/data/00005-5-b783c797-959e-4b9c-bd1c-dcb4aa30f3d7-0-00001.parquet. This is unsupported
                                                                                

+--------+--------+------+
|order_id|customer|amount|
+--------+--------+------+
|       1|   Alice| 100.5|
|       4|   David| 420.0|
|       2|     Bob|230.75|
|       5|     Eva| 180.5|
|       3| Charlie| 315.2|
+--------+--------+------+



In [20]:
entry = next(client.get_log("main"))
print(entry)
print(dir(entry))


LogEntry(commit_meta=CommitMeta(hash_='636d8430c9628ac233a3ec88a814921e4122ff2c6767c38dafeea66a5353a617', commitTime=datetime.datetime(2025, 10, 21, 7, 31, 3, 708641, tzinfo=datetime.timezone.utc), authorTime=datetime.datetime(2025, 10, 21, 7, 31, 3, 708641, tzinfo=datetime.timezone.utc), committer='', author='root', signedOffBy=None, message='Iceberg append against demo_db.sales', properties={'app-id': 'local-1761031120340', 'iceberg.operation': 'append', 'application-type': 'iceberg'}), parent_commit_hash='337cba8ab76111ea4f24b0a0c96255dad18fec0d8c48a44d6cc981c37b0673db', operations=None)
['__annotations__', '__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '

In [22]:
for log_entry in client.get_log("main"):
    print(f"Commit hash: {log_entry.__hash__} | Message: {log_entry.commit_meta.message}")


Commit hash: None | Message: Iceberg append against demo_db.sales
Commit hash: None | Message: Iceberg append against demo_db.sales
Commit hash: None | Message: Iceberg table created/registered with name demo_db.sales
Commit hash: None | Message: create namespace demo_db


In [23]:
client.create_branch("experiment", "main")
print("✅ Branch 'experiment' created from 'main'")


NessieException: Client Error Bad Request: Target hash must be provided.

In [ ]:
# Tạo SparkSession trỏ tới branch 'experiment'
spark_branch = (
    SparkSession.builder.appName("NessieExperimentBranch")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "experiment")   # 👈 Chuyển branch
    .config("spark.sql.catalog.nessie.warehouse", "s3a://warehouse/")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .getOrCreate()
)

# Ghi thêm dữ liệu chỉ trong branch 'experiment'
data3 = [(6, "Frank", 999.9)]
spark_branch.createDataFrame(data3, ["order_id", "customer", "amount"])\
    .writeTo("nessie.demo_db.sales").append()

print("✅ Wrote data to branch 'experiment'")


In [ ]:
spark.sql("SELECT * FROM nessie.demo_db.sales ORDER BY order_id").show()


In [ ]:
spark_branch.sql("SELECT * FROM nessie.demo_db.sales ORDER BY order_id").show()


In [32]:
spark.sql("SELECT * FROM nessie.demo_db.sales.snapshots").show(truncate=False)

+-----------------------+-------------------+------------------+---------+------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id         |operation|manifest_list                                                                                                                                   |summary                                                                                                

In [35]:
snapshot_id = 1791065480393736880

query = f"""
SELECT * FROM nessie.demo_db.sales VERSION AS OF {snapshot_id}
"""
old_df = spark.sql(query)
old_df.show()


+--------+--------+------+
|order_id|customer|amount|
+--------+--------+------+
|       1|   Alice| 100.5|
|       2|     Bob|230.75|
|       3| Charlie| 315.2|
|       4|   David| 420.0|
|       5|     Eva| 180.5|
+--------+--------+------+



In [29]:
entry = next(client.get_log("main"))
print(entry)
print(dir(entry))


LogEntry(commit_meta=CommitMeta(hash_='636d8430c9628ac233a3ec88a814921e4122ff2c6767c38dafeea66a5353a617', commitTime=datetime.datetime(2025, 10, 21, 7, 31, 3, 708641, tzinfo=datetime.timezone.utc), authorTime=datetime.datetime(2025, 10, 21, 7, 31, 3, 708641, tzinfo=datetime.timezone.utc), committer='', author='root', signedOffBy=None, message='Iceberg append against demo_db.sales', properties={'app-id': 'local-1761031120340', 'iceberg.operation': 'append', 'application-type': 'iceberg'}), parent_commit_hash='337cba8ab76111ea4f24b0a0c96255dad18fec0d8c48a44d6cc981c37b0673db', operations=None)
['__annotations__', '__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '